In [1]:
import os
os.environ["DEBUG"] = "1"

from tinygrad.tensor import Tensor
from tinygrad.helpers import dtypes
from tinygrad.ops import Device
Device.DEFAULT = "CPU"

import numpy as np
from typing import List, Dict, Tuple, Union, Optional
import math
import torch

## Steps

1. Learn about the multinomial distribution
2. experiment with torch.multinomial
   1. only takes ndim = 1 or 2
3. write some psuedocode for tinygrad
4. get a brute force implementation working with tinygrad
5. write tests
6. optimize


### What is the multinomial distribution?
A generalization of the binomial distribution, there are $n$ trials and each trial results in one of $k$ possible outcomes, with each outcome having a cerain probability.
- in each of the $n$ trials, one of $k$ outcomes occurs
- the probability of each outcome is specified by a vector $\vec{p}$ of length $k$, where the sum of the elements of $\vec{p}$ is 1 (as it is a probability distribution)
- the resulting distribution gives the probability of each combinaiton of outcomes over all $n$ trials

### Pytorch's Implementation


### Psuedocode

```
- input
    - input tensor (ndim=1 or ndim=2)
    - n is the number of samples to draw
    - replacement (bool) whether to draw with replacement or not
- validate input
    - make sure input is a tensor of ndim=1 or ndim=2, else raise error
    - make sure there's no negative values in the input tensor, else raise error
    - make sure there are no null values in the input tensor, else raise error
    - make sure the sum of the input tensor is 1, else raise error (or normalize it)
- errors
    - if replacement is False, and n > k (len of input tensor), raise error 
```



## Let's play with torch multinomial

In [ ]:
# torch.multinomial(input, num_samples, replacement=False, *, generator=None, out=None) → LongTensor

#  Returns a tensor where each row contains num_samples indices sampled from the multinomial probability distribution
#  located in the corresponding row of tensor input.

In [136]:
example_1d = torch.tensor([.2, .4, .4])
example_1d_multinomial = torch.multinomial(example_1d, 3, replacement=True)
print(example_1d_multinomial)

example_2d = torch.tensor([[.2, .4, .4], [.3, .3, .4]])
example_2d_multinomial = torch.multinomial(example_2d, 5, replacement=True)
print(example_2d_multinomial)

tensor([0, 0, 1])
tensor([[1, 2, 1, 2, 1],
        [0, 0, 1, 1, 2]])


## looking at the old PR

In [ ]:
@staticmethod
def choice(a: Union[Tensor, int], size: Union[int, Tuple[int, ...]] = 1, p: Optional[Tensor] = None):
    if isinstance(a, int): a = Tensor.arange(a)
    assert isinstance(a, Tensor) and a.ndim == 1, "a must be 1-dimensional"
    if p is not None:
        assert a.shape == p.shape, "a and p must have the same shape"
    else:
        p = Tensor.full(a.shape, 1 / a.numel())
    size = (size,) if isinstance(size, int) else size
    cdf = p.cumsum()
    cdf /= cdf[-1] # probabilities should sum to 1
    unif_samples = Tensor.rand(n_samples := math.prod(size), 1)
    indices = (unif_samples.expand(n_samples, a.numel()) >= cdf).sum(1).cast(dtypes.int32)
    return a[indices].reshape(size)

## Psuedocode

```
- input
    - input tensor (ndim=1 or ndim=2)
    - n is the number of samples to draw
    - replacement (bool) whether to draw with replacement or not
- validate input
    - make sure input is a tensor of ndim=1 or ndim=2, else raise error
    - make sure there's no negative values in the input tensor, else raise error
    - make sure there are no null values in the input tensor, else raise error
    - make sure the sum of the input tensor is 1, else raise error (or normalize it - pytorch does not automatically normalize i think)
- errors
    - if replacement is False, and n > k (len of input tensor), raise error 
```

```
function multinomial(input_tensor, n_samples, replacement)
    validate input
    check replacement logic, raise error if necessary

    if 2D tensor, iterate over each row:
        normalize if necessary
        calculate the CDF
        get the random numbers
        convert the random numbers to indices based on the CDF
        return the indices
    else if 1D tensor:
        normalize if necessary
        calculate the CDF
        get the random numbers
        convert the random numbers to indices based on the CDF
        return the indices
```

```
multinomial(input_tensor, n_samples, replacement)
    validation logic
    replacment logic
    def multinomial_1d(input_tensor of ndim=1, n_samples, replacement)
        normalize if necessary
        calculate the CDF
        get the random numbers
        convert the random numbers to indices based on the CDF
        return the indices

    if 2d:
        for each row:
            multinomial_1d(row, n_samples, replacement)
    else:
        multinomial_1d(input_tensor, n_samples, replacement)

    return the indices
```

In [200]:
@staticmethod
def multinomial(input: Tensor, num_samples: int, replacement: bool = False):
    assert isinstance(input, Tensor) and (input.ndim == 1 or input.ndim == 2), "input must be a 1 or 2-dimensional tensor"
    assert isinstance(num_samples, int) and num_samples > 0, "num_samples must be a positive integer"
    if replacement == False:
        assert num_samples <= input.numel(), f"num_samples: {num_samples} must be less than or equal to the number of elements in input tensor: {input.numel()}"
        # TODO: implement without replacement
        raise NotImplementedError("multinomial without replacement not implemented")
    
    input = input.reshape(-1, 1) if input.ndim == 1 else input # handling 1D and 2d input uniformly
    
    cdf = input.cumsum() # cumulative distribution function
    cdf /= cdf[:, -1, None] # normalize each row of cdf to sum to 1)
    unif_samples = Tensor.rand(num_samples, 1) if input.ndim == 1 else Tensor.rand(input.shape[0], num_samples, 1)
    cdf_expanded = cdf.unsqueeze(-1) if input.ndim == 1 else cdf.unsqueeze(2) # Expanding dimensions for broadcasting
    # Broadcasting comparison
    indices = (unif_samples >= cdf_expanded).sum(dim=-1) - 1
    indices = indices.cast(dtypes.int32)


    # rest of logic here
    return indices.flatten() if input.ndim == 1 else indices

Tensor.multinomial = multinomial

In [254]:
# Let's make test cases and play with the logic/try to clean it up. can use pytorch for help
p_1d = Tensor([.2, .2, .4])
p_2d = Tensor([[.2, .4, .4], [.3, .3, .4]])
p_3d = Tensor([[[.2, .4, .4], [.3, .3, .4]], [[.2, .4, .4], [.3, .3, .4]]])

num_samples = 5
replacement = True

cdf_1d = p_1d.cumsum() 
cdf_1d /= cdf_1d[-1] # probabilities should sum to 1
assert cdf_1d[-1] == 1, "probabilities should sum to 1"
# print(cdf_1d.numpy())

cdf_2d = p_2d.cumsum()
cdf_2d /= cdf_2d[:, -1, None] # normalize each row of cdf to sum to 1)
assert all(cdf_2d[:, -1] == 1), "probabilities should sum to 1"
print(cdf_2d.numpy())

[[0.5        1.         1.        ]
 [0.625      0.87500006 1.        ]]


In [275]:
# 1D case
p_1d = Tensor([.2, .2, .4])
num_samples = 5

def _1D(input: Tensor, num_samples: int, replacement=False):
    assert input.ndim == 1, "only takes 1d input"
    cdf = input.cumsum()
    cdf /= cdf[-1] # probabilities should sum to 1
    unif_samples = Tensor.rand(num_samples, 1) # uniform samples
    indices = (unif_samples >= cdf).sum(1) # find the first index where the unif_samples are greater than the cdf


cdf_1d = p_1d.cumsum()
cdf_1d /= cdf_1d[-1] # probabilities should sum to 1
print(cdf_1d.numpy())
unif_samples = Tensor.rand(num_samples, 1)
print(unif_samples.numpy())
# find the first index where the unif_samples are greater than the cdf
indices = (unif_samples >= cdf_1d).sum(1)
print(indices.numpy())

[0.25 0.5  1.  ]
[[0.32764024]
 [0.5309934 ]
 [0.9486966 ]
 [0.5760423 ]
 [0.1702497 ]]
[1. 2. 2. 2. 0.]


In [250]:
print(matrix[:, -1]) # last el of each row
# get each row
# take the matrix and divide each row by the last element in the row
print(matrix / matrix[:, -1, None])

tensor([3, 6])
tensor([[0.3333, 0.6667, 1.0000],
        [0.6667, 0.8333, 1.0000]])


In [144]:
t = Tensor([.2, .4, .1, .3])
cdf = t.cumsum()
print(cdf.numpy())

[0.2        0.6        0.70000005 1.        ]


In [117]:
t = Tensor([[0.1, 0.2, 0.3, 0.4], [0.4, 0.3, 0.2, 0.1]])
print(t.multinomial(5, replacement=True).numpy())

[[0 0 0 0 0]
 [0 0 0 0 0]]


In [105]:
t = Tensor.rand(3, 4)
res = t.multinomial(10)

In [106]:
res.numpy()

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [107]:
t = Tensor.rand(3,4)
for i in range(t.shape[0]):
    row = t[i]
    print(row.shape)
    row_prob = row / row.sum()
    print(row_prob.shape)
    cum_dist = row_prob.cumsum()

(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
